### Importing stuff

In [ ]:
# baseline cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

### Loading the data set
* greyscale image 
* encoding 

In [ ]:
def load_dataset():
    (x_train , y_train) , (x_test , y_test) = mnist.load_data()
    # grayscale image 
    trainX = x_train.reshape((x_train.shape[0], 28, 28, 1))
    testX = x_test.reshape((x_test.shape[0], 28, 28, 1))
    print(x_train.shape[0])
    # one hot encoding the target values
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return x_train, x_test, y_train, y_test

### Normalizing the data
* int to float
* 0 to 1 range for all pixels

In [ ]:
def prep_pixels(train , test):
    # convert from int to float
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalization to 0-1 range
    train_norm /= 255.0
    test_norm /= 255.0
    return train_norm , test_norm

### Plotting the handwritten digits
* so we should try plotting a graph of all the numbs in the dataset using a function

In [ ]:
 x_train, x_test, y_train, y_test=load_dataset()
fig, axes = plt.subplots(4,4,figsize=(5,5))
for i in range(16):
    # plt.subplot(330 + 1 + i) #330 means that 33 means 3x3 grid and then the postions are added 
    #plt.imshow(x_train[i], cmap = plt.get_cmap('gray'))
    # a better to do the sam thing is 
    axes[i//4,i%4].imshow(x_train[i] , cmap = plt.get_cmap('gray'))
    axes[i//4, i%4].axis('off')
print(len(y_train))

plt.show()

In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation = 'relu', kernel_initializer='he_uniform', input_shape = (28,28,1)))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(100,activation='relu',kernel_initializer='he_uniform'))
    model.add(Dense(10,activation='softmax'))
    #compiling the model
    opt = SGD(learning_rate=0.01,momentum=0.9)
    model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def evaluate_model(dataX, dataY, n_folds=10):
    kfold = KFold(n_splits=n_folds, shuffle=True)
    scores = []
    histories = []
    print("Shape of dataX:", dataX.shape)
    print("Shape of dataY:", dataY.shape)  # Add this line to check the shape of dataY
    
    for train_ix, test_ix in kfold.split(dataX):
        model = define_model()
        
        print("Train indices:", train_ix)  # Add this line to check train_ix
        print("Test indices:", test_ix)    # Add this line to check test_ix
        
        trainX, testX = dataX[train_ix], dataX[test_ix]
        trainY, testY = dataY[train_ix], dataY[test_ix]
        
        history = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=32, verbose=0)
        _, acc = model.evaluate(testX, testY, verbose=0)
        
        scores.append(acc)
        histories.append(history)
    return scores, histories


In [ ]:
def summarize_diagnositics(histories):
    for i in range(len(histories)):
        # plotting loss
        plt.subplot(2,1,1)
        plt.title('Cross entropy Loss')
        plt.plot(histories[i].histories['loss'], color = 'blue', label= 'train')
        plt.plot(histories[i].histories['val_loss'], color='red', label = 'test')
        # plotting accuracy
        plt.subplot(2,1,2)
        plt.title('Classification Accuracy')
        plt.plot(histories[i].history['accuracy'], color = 'blue', label= 'train')
        plt.plot(histories[i].history['val_accuracy'], color = 'red', label= 'test')
        plt.show()

In [ ]:
def summarize_performance(scores):
    print("Accuracy: mean=%.3f std=%.3f, n=%d" % (mean(scores)*100 , std(scores)*100 , len(scores)))
    plt.boxplot(scores)
    plt.show()

In [ ]:
def run_test_harness():
    trainX, testX, trainY, testY = load_dataset()
    trainX, testX = prep_pixels(trainX, testX)
    scores, histories = evaluate_model(trainX, trainY)
    summarize_diagnositics(histories)
    summarize_performance(scores)

In [ ]:
run_test_harness()